# Setup collections and indexes with PhariaSearch

While large language models come equipped with extensive built-in knowledge, answering questions given a known text may not be sufficient for your use case. At some point, you will probably want to search through, or answer questions about, your own knowledge base.

You can leverage Aleph Alpha's PhariaSearch (previously known as DocumentIndex) – a robust semantic search tool – to pinpoint sections in documents that align closely with your query.

In this tutorial, we will go through the creation of Collections and Index configurations, and the upload of text directly to PhariaSearch.

## Environment Validation

⚠️ **CRITICAL: Validate your environment before proceeding**

This step ensures your `.env` file is properly configured with valid API endpoints, authentication tokens, and unique resource names.

**DO NOT SKIP!** If validation fails:
- Check your `.env` file for missing values
- Contact your infrastructure administrator if errors persist

Only proceed after ALL checks pass.

In [ ]:
from validation_utils import validate_environment

# Run environment validation
validate_environment()

# If validation fails, DO NOT proceed with the rest of the tutorial
# Fix the issues identified above first!

### Load Environment Variables

  Now we'll load all the configuration parameters from your `.env` file. These include API endpoints, authentication tokens, and the names of resources
  we'll create in PhariaSearch.

In [ ]:
from os import getenv
from dotenv import load_dotenv

load_dotenv(override=True)

# Please make sure to include the /v1 in the api base url
PHARIA_API_BASE_URL = getenv("PHARIA_API_BASE_URL")
PHARIA_SEARCH_API_URL = f"{PHARIA_API_BASE_URL}/v1/studio/search"
TOKEN = getenv("PHARIA_AI_TOKEN")

NAMESPACE = getenv("PHARIA_DATA_NAMESPACE")
COLLECTION = getenv("PHARIA_DATA_COLLECTION")
INDEX = getenv("INDEX")
HYBRID_INDEX = getenv("HYBRID_INDEX")
FILTER_INDEX = getenv("FILTER_INDEX")

EMBEDDING_MODEL_NAME = getenv("EMBEDDING_MODEL_NAME")

print("\n")
print("\033[92m✅ SUCCESS: Environment variables loaded\033[0m")
print(f"   • Namespace: {NAMESPACE}")
print(f"   • Collection: {COLLECTION}")
print(f"   • API Base URL: {PHARIA_API_BASE_URL}")
print("\n")

## 1. Setup a Search Client with Pharia Data SDK

To search through PhariaSearch, you'll first need to setup the Search Client using the environment variables we specified at the beginning of the notebook.

In [ ]:
from pharia_data_sdk.connectors import DocumentIndexClient

search_client = DocumentIndexClient(
    token=TOKEN,
    base_url=PHARIA_SEARCH_API_URL,
)

print("\n")
try:
    print(f"\033[92m✅ SUCCESS: Search client created\033[0m")
    print(f"   • Server URL: {search_client._base_url}")
except AttributeError:
    print("\033[92m✅ SUCCESS: Search client created\033[0m")
print("\n")

## 2. Create a collection
The collection is the place where our searchable content will be stored. The following code will user the Pharia Data SDK to create a new collection with the specified details.

In [ ]:
from pharia_data_sdk.connectors.document_index.document_index import CollectionPath
from time import sleep

collection_path = CollectionPath(namespace=NAMESPACE, collection=COLLECTION)

try:
    search_client.delete_collection(collection_path)
    
    # Wait in loop for at max 10 seconds for the collection to be deleted. Poll every 0.5 seconds.
    max_wait_time = 10.0
    poll_interval = 0.5
    elapsed_time = 0.0
    
    print("\n")
    print("\033[93m⚙️  PROCESSING: Waiting for collection deletion...\033[0m")
    
    while elapsed_time < max_wait_time:
        try:
            # Try to get the collection - if it exists, continue waiting
            search_client.documents(collection_path)
            sleep(poll_interval)
            elapsed_time += poll_interval
        except Exception:
            # Collection is deleted when we can't access it anymore
            print("\033[92m✅ SUCCESS: Collection successfully deleted\033[0m")
            break
    else:
        # Timeout reached
        print(f"\033[93m⚙️  PROCESSING: Timeout after {max_wait_time}s, proceeding anyway\033[0m")
    
except Exception as e:
    print(f"\033[94mℹ️  INFO: Collection deletion: {e}\033[0m")
    pass

search_client.create_collection(collection_path)

print("\n")
print("\033[92m✅ SUCCESS: Collection created/retrieved\033[0m")
print(f"   • Collection: {COLLECTION}")
print(f"   • Namespace: {NAMESPACE}")
print("\n")

## 3. Create a semantic index configuration

Now, let's create an index and assign it to this collection. You can do this before or after populating the collection with documents; PhariaSearch automatically updates semantic indexes in the background.
In this tutorial we use `pharia-1-embedding-256-control` embedding model. If this model is not available in the environment, please, replace the name with the one that is available (e.g. `luminous-base`).

In [ ]:
from pharia_data_sdk.connectors.document_index.document_index import (
    IndexConfiguration,
    IndexPath,
    SemanticEmbed,
)

index_path = IndexPath(namespace=NAMESPACE, index=INDEX)

# customise the parameters of the index here
index_configuration = IndexConfiguration(
    chunk_size=64,
    chunk_overlap=0,
    embedding=SemanticEmbed(model_name=EMBEDDING_MODEL_NAME, representation="asymmetric"),
)

search_client.create_index(index_path, index_configuration)

print("\n")
print("\033[92m✅ SUCCESS: Semantic index created\033[0m")
print(f"   • Index: {INDEX}")
print(f"   • Model: {EMBEDDING_MODEL_NAME}")
print("\n")

# assign the index to the collection
search_client.assign_index_to_collection(collection_path, INDEX)

print("\033[92m✅ SUCCESS: Index assigned to collection\033[0m")
print(f"   • Collection: {COLLECTION}")
print("\n")

## 4. Upload some text to the collection

Now that we have our collection set up, we need to populate it with content that can be searched. Let's create three text objects that will serve as our sample knowledge base. These documents will demonstrate how PhariaSearch can identify relevant information across different types of content.

We'll add biographical information about notable figures to showcase the semantic search capabilities. The document content is defined in `sample_documents.py` file, outside of the notebook and imported below.

We'll upload each document to our collection. The SDK will automatically handle the document storage and prepare them for indexing.

In [ ]:
from pharia_data_sdk.connectors.document_index.document_index import (
    DocumentContents,
    DocumentPath,
)

from sample_documents import documents

print("\n")
print("\033[93m⚙️  PROCESSING: Uploading documents to collection\033[0m")
print("\n")

for doc in documents:
    document_path = DocumentPath(
        collection_path=collection_path, document_name=doc["name"]
    )
    search_client.add_document(
        document_path, contents=DocumentContents.from_text(doc["content"])
    )
    print(f"   ✓ Document '{doc['name']}' uploaded")

print("\n")
print(f"\033[92m✅ SUCCESS: All {len(documents)} documents uploaded\033[0m")
print("\n")

  ### Verify Documents in Collection

  Let's verify that our documents have been successfully uploaded to the collection:

In [ ]:
from local_utils import print_documents_with_time

docs = search_client.documents(collection_path)
print_documents_with_time(docs, COLLECTION)

Once the text is indexed, we can also have a look at its chunks:

In [ ]:
from pharia_data_sdk.connectors import ResourceNotFound
from local_utils import print_document_chunks

print("\n")
print("\033[94mℹ️  INFO: Checking document chunks (if indexed)\033[0m")
print("\n")

try:
    chunks = search_client.chunks(
        DocumentPath(collection_path=collection_path, document_name=documents[0]["name"]),
        index_name=INDEX,
    )
    print(f"\033[92m✅ SUCCESS: Found {len(chunks)} chunks for document '{documents[0]['name']}'\033[0m")
    print_document_chunks(chunks, documents[0]["name"])
except ResourceNotFound:
    print("\033[93m⚙️  PROCESSING: Document still embedding, try again in a couple of minutes\033[0m")
    pass  # This is expected if the document is still embedding.
    
print("\n")

## 5. Setup search indexes

### 5.1. Perform semantic search

Now that we have uploaded our text, we can search through it using the semantic similarities between a given query and each chunk.

To do so, let's use the `DocumentIndexRetriever`:

In [ ]:
from pharia_data_sdk.connectors.retrievers import DocumentIndexRetriever
from local_utils import print_search_results

document_index_retriever = DocumentIndexRetriever(
    document_index=search_client,
    index_name=INDEX,
    namespace=NAMESPACE,
    collection=COLLECTION,
    k=12,
    threshold=0.55
)

query = "The influence of Robert Moses"

print("\n")
print(f"\033[94mℹ️  INFO: Testing semantic search with query '{query}'\033[0m")
print("\033[93m⚙️  PROCESSING: Search in progress, please wait...\033[0m")
print("\n")

results = document_index_retriever.get_relevant_documents_with_scores(query=query)

print_search_results(results, query)

### 5.2 Hybrid Search

PhariaSearch supports hybrid search, which combines results of semantic search and keyword search.
In order to use hybrid search, we need to create a hybrid index and assign it to the collection:

In [ ]:
index_path = IndexPath(namespace=NAMESPACE, index=HYBRID_INDEX)

# customise the parameters of the index here
index_configuration = IndexConfiguration(
    chunk_size=64,
    chunk_overlap=0,
    hybrid_index="bm25",
    embedding=SemanticEmbed(model_name=EMBEDDING_MODEL_NAME, representation="asymmetric"),
)

# create the namespace-wide index resource
index = search_client.create_index(index_path, index_configuration)

print("\n")
print("\033[92m✅ SUCCESS: Hybrid index created\033[0m")
print(f"   • Index: {HYBRID_INDEX}")
print(f"   • Type: BM25 + Semantic")
print("\n")

# assign the index to the collection
search_client.assign_index_to_collection(collection_path, HYBRID_INDEX)

print("\033[92m✅ SUCCESS: Hybrid index assigned to collection\033[0m")
print("\n")

In [ ]:
search_client.list_indexes(NAMESPACE)

If we now search on the hybrid index, we will not only get chunks with a semantic similarity but also chunks that match the keywords in the query:

In [ ]:
from local_utils import print_search_results

document_index_retriever = DocumentIndexRetriever(
    document_index=search_client,
    index_name=HYBRID_INDEX,
    namespace=NAMESPACE,
    collection=COLLECTION,
    k=5,
    threshold=0.5,
)

query = "25 April"

print("\n")
print(f"\033[94mℹ️  INFO: Testing hybrid search with query '{query}'\033[0m")
print("\033[93m⚙️  PROCESSING: Search in progress, please wait...\033[0m")
print("\n")

results = document_index_retriever.get_relevant_documents_with_scores(query=query)

print_search_results(results, query)

### 5.3  Search with Metadata filtering

PhariaSearch also supports filter-indexes, which gives us the ability to provide specific filters in case we want to filter our search based on each document's metadata.

To do so, let's first upload another version of our documents but this time with some metadata e.g. the "title" field.

In [ ]:
print("\n")
print("\033[93m⚙️  PROCESSING: Re-uploading documents with metadata\033[0m")
print("\n")

for doc in documents:
    document_path = DocumentPath(
        collection_path=collection_path, document_name=doc["name"]
    )
    search_client.add_document(
        document_path,
        contents=DocumentContents(
            contents=[doc["content"]], metadata={"title": doc["name"]}
        ),
    )
    print(f"   ✓ Document '{doc['name']}' uploaded with metadata")

print("\n")
print(f"\033[92m✅ SUCCESS: All {len(documents)} documents uploaded with metadata\033[0m")
print("\n")

#### Preparation of the Filter Index
To be able to use metadata filtering, we need to first check the following:
1. Check if we already have a search index assigned. If not, we need to assign one because filter-indexes can be defined at the namespace level but can only be assigned to already existing search indexes 
2. Define a new filter-index configuration for our specific collection metadata.
3. Assign the filter-index that we created to a search index.

In [ ]:
# 1
# list all the assigned search indexes for our collection
search_client.list_assigned_index_names(collection_path=collection_path)

In [ ]:
# 2
# define a new filter-index
search_client.create_filter_index_in_namespace(
    namespace=collection_path.namespace,
    filter_index_name=FILTER_INDEX,  # this is how our filter-index is identified in our namespace
    field_name="title",  # this is the name of the field to which we want to apply our filter
    field_type="string",  # type of the field we want to apply our filter to. Must be one of "string", "integer", "float", "boolean" or "datetime"
)

# let's check if our index is present now
filter_present = FILTER_INDEX in search_client.list_filter_indexes_in_namespace(
    namespace=collection_path.namespace
)

print("\n")
if filter_present:
    print(f"\033[92m✅ SUCCESS: Filter index created\033[0m")
    print(f"   • Filter: {FILTER_INDEX}")
    print(f"   • Field: title (string)")
else:
    print(f"\033[91m⚠️  ERROR: Filter index creation failed\033[0m")
print("\n")

In [ ]:
# 3
# assign our new filter-index to our collection
search_client.assign_filter_index_to_search_index(
    collection_path=collection_path,
    index_name=INDEX,  # we assign it to intelligence-layer-sdk-demo-index
    filter_index_name=FILTER_INDEX,
)

# check if our filter-index is assigned to our collection
assigned_filters = search_client.list_assigned_filter_index_names(
    collection_path=collection_path, index_name=INDEX
)

print("\n")
print(f"\033[92m✅ SUCCESS: Filter index assigned to collection\033[0m")
print(f"   • Assigned filters: {assigned_filters}")
print("\n")

 Now, as we have the filter-index enabled, we need to initialize a new `DocumentIndexRetriever` with the search index for which we added the filter-index.

In [ ]:
document_index_retriever = DocumentIndexRetriever(
    document_index=search_client,
    index_name=INDEX,
    namespace=NAMESPACE,
    collection=COLLECTION,
    k=5,
)

print("\n")
print(f"\033[94mℹ️  INFO: Document retriever configured for filtered search\033[0m")
print(f"   • Index: {INDEX}")
print(f"   • Collection: {COLLECTION}")
print("\n")

#### Defining and Using a Filter
Before we perform the filtered search we have to define a filter.
Filters are composed of the following elements:
- `filter_type` which can be one of "with", "without" or "with_one_of"
- `filter_fields`, which defines the actual filtering criteria over a certain value for our chosen field

If we want a filter that accepts only documents with the value of the "title" field equal to the "name" field of `document_1`, we define the filter as follows:

In [ ]:
from pharia_data_sdk.connectors import FilterField, FilterOps, Filters

filters = Filters(
    filter_type="with",  # we want to only return documents matching our filter
    fields=[
        FilterField(
            field_name="title",  # this is the key we used in our metadata dict
            field_value=documents[0]["name"],  # this is what we used as a value in the metadata dict
            criteria=FilterOps.EQUAL_TO,  # we want to match exactly
        ),
    ],
)

print("\n")
print(f"\033[94mℹ️  INFO: Filter created for documents with title='{documents[0]['name']}'\033[0m")
print("\n")

In [ ]:
# let's use the filters with our query to restrict the search to documents with the title "robert_moses"
from local_utils import print_search_results

query = "Robert Moses"

print("\n")
print(f"\033[94mℹ️  INFO: Testing filtered search with query '{query}'\033[0m")
print(f"   • Filter: Only documents with title='{documents[0]['name']}'")
print("\033[93m⚙️  PROCESSING: Search in progress, please wait...\033[0m")
print("\n")

results = document_index_retriever.get_relevant_documents_with_scores(
    query=query, filters=[filters]
)

print_search_results(results, f"{query} (filtered)")

Great! We only get document with the "robert_moses" as the exact title in the metadata

#### Exclusion of Documents
We can also modify our filter such that we only get documents that do *not* match the specified filter fields. This is as simply as replacing the "with" filter_type with a "without": 

In [ ]:
# let's now try to exclude the document with the title "robert_moses"
filters_without = Filters(
    filter_type="without",  # we change this to "without" to exclude the document
    fields=[
        FilterField(
            field_name="title",  # this is the key we used in our metadata dict
            field_value=documents[0]["name"],  # this is what we used as a value in the metadata dict
            criteria=FilterOps.EQUAL_TO,  # we want to match exactly
        ),
    ],
)

print("\n")
print(f"\033[94mℹ️  INFO: Filter created to EXCLUDE documents with title='{documents[0]['name']}'\033[0m")
print("\n")

In [ ]:
# let's use the filters with our query to exclude the document with the title "robert_moses"
from local_utils import print_search_results

query = "Robert Moses"

print("\n")
print(f"\033[94mℹ️  INFO: Testing exclusion filter with query '{query}'\033[0m")
print(f"   • Filter: Exclude documents with title='{documents[0]['name']}'")
print("\033[93m⚙️  PROCESSING: Search in progress, please wait...\033[0m")
print("\n")

results = document_index_retriever.get_relevant_documents_with_scores(
    query=query, filters=[filters_without]
)

print_search_results(results, f"{query} (excluding filtered)")

Notice, how we only get results where the `document_name` is not "robert_moses". 

## 6. Conclusions

We have now a collection with some documents uploaded and with semantic indexing, hybrid indexing, and the possibility to use metadata for filtering results.